In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import re
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

In [2]:

# Scrolling to the bottom to load all content
def scrolldown():
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        body = driver.find_element(By.TAG_NAME, "body")
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False
                

In [3]:
#GETTING ROUTE NAMES AND ROUTE_LINKS

def route_name_link(link):

   driver = webdriver.Chrome()
   wait = WebDriverWait(driver, 10)
   driver.get(link)
   driver.implicitly_wait(3)    
   route_and_link=[]

   pc = wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="DC_117_paginationTable"]')))
   num_pgs =len(driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs "))+1
   if num_pgs==1:
      #getting bus routes names
      Bus_routes=driver.find_elements(By.XPATH,"//a[@class='route']")
      allroute=[i.text for i in Bus_routes]
      
      #getting bus_routes_links
      nested_div=wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.D117_main.D117_container')))
      route_links = nested_div.find_elements(By.CSS_SELECTOR, 'div.route_link')
      link=[i.find_element(By.TAG_NAME,"a").get_attribute('href') for i in route_links]
      
      r=zip(allroute,link)
      route_and_link.extend(r)
   else:
      for i in range(1,num_pgs):
         #getting bus routes names
         Bus_routes=driver.find_elements(By.XPATH,"//a[@class='route']")
         allroute=[i.text for i in Bus_routes]  
         
         #getting bus_routes_links
         nested_div=wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.D117_main.D117_container')))
         route_links = nested_div.find_elements(By.CSS_SELECTOR, 'div.route_link')
         link=[i.find_element(By.TAG_NAME,"a").get_attribute('href') for i in route_links]
         
         r=zip(allroute,link)
         route_and_link.extend(r)
         
         if i==num_pgs-1:
            break
         else:
            next_page= pc.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{i+ 1}"]')
            action = ActionChains(driver)
            action.move_to_element(next_page).perform()
            time.sleep(2)
            next_page.click()
            time.sleep(3)
      
   return route_and_link

In [4]:
#click view buses
def click_view_buses():
    try:
        # Locate and click the 'View Buses' button if needed
            view_buses_button = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button']")))
            #print("len()", len(view_buses_button))
            for i in range(len(view_buses_button)-1,-1,-1):
                view_buses_button[i].click()
                time.sleep(2)
    except:
            pass
            for view_buses in view_buses_button:
                try:
                    view_buses.click()
                    scrolling = True
                    while scrolling:
                        old_page_source = driver.page_source
                        body = driver.find_element(By.TAG_NAME, "body")
                        body.send_keys(Keys.PAGE_DOWN)
                        time.sleep(2)
                        new_page_source = driver.page_source
                        if new_page_source == old_page_source:
                            scrolling = False
                    time.sleep(5)
                except Exception as e:
                    print(e)
    time.sleep(5)

In [5]:
#getting bus details

def get_bus(link):
    driver.get(link)
    time.sleep(2)
    click_view_buses()
    #scrolldown()
    b=driver.find_elements(By.CSS_SELECTOR,"div[class='button']")
    for i in range(len(b)-1,-1,-1):
        try:
            b[i].click()
        except:
            print()
    x=0
    l=[]
    while True:
        x=x+1
        driver.execute_script('scrollBy(0,100)')
        #time.sleep(0.5)
        if(x>1000):
            break
    details = []
    bus_containers = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'clearfix bus-item')]")))
    for bus_container in bus_containers:
        try:
            
            #routes_name = bus_container.find_element(By.XPATH, ".//div[@class='D136_h1']").text
            busname = bus_container.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
            bustype = bus_container.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
            departuretiming = bus_container.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
            duration = bus_container.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
            reachtiming = bus_container.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
            rating = bus_container.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
            price = bus_container.find_element(By.XPATH, ".//div[@class='fare d-block']").text
            seat = bus_container.find_element(By.XPATH, ".//div[@class='column-eight w-15 fl']").text

            details.append({
                'links':link,
                #'Route_name': routes_name,
                'Bus_name': busname,
                'Bus_type': bustype,
                'Departure_time': departuretiming,
                'Duration': duration,
                'Arrival_time': reachtiming,
                'Ratings': rating,
                'Prices': price,
                'Seat_availability': seat,
            })
        except:
            pass     
    return details

In [6]:
def flatten_list(nested_list):
    """Flattens a nested list."""
    return [item for sublist in nested_list for item in sublist]

In [7]:
#to get all states data in Redbus
def all_state(state_link): 
    name_rlink= route_name_link(state_link)
    driver = webdriver.Chrome()
    route_link=[i[1] for i in name_rlink]

    all_data=[]
    for link in route_link:
        try:
            all_data.append(get_bus(link))
            driver = webdriver.Chrome()
        except:
            pass
            

# Flattening the collected data
    flattened_data = flatten_list(all_data)
    df=pd.DataFrame(flattened_data)
    return df

In [8]:
#main code

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
action = ActionChains(driver)

AP_data =all_state('https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile')
Kerala_data=all_state('https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile')
jammukashmir_data=all_state('https://www.redbus.in/online-booking/jksrtc')
punjab_data=all_state('https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile')
Rajasthan_data=all_state('https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile')
Telangana_data=all_state('https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile')
West_bengal_data=all_state('https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtch')
karbi_anglong_data=all_state('https://www.redbus.in/online-booking/kaac-transport')
Kadamba_data= all_state('https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile')
Bihar_data=all_state('https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile')

#splitting route from links
AP_data_df = AP_data
AP_data_df['route'] = AP_data['links'].apply(lambda x: x.split('/')[-1])

Kerala_data_df = Kerala_data
Kerala_data_df['route'] = Kerala_data['links'].apply(lambda x: x.split('/')[-1])

jammukashmir_data_df=jammukashmir_data
jammukashmir_data_df['route'] = jammukashmir_data['links'].apply(lambda x: x.split('/')[-1])

punjab_data_df=punjab_data
punjab_data_df['route'] = punjab_data['links'].apply(lambda x: x.split('/')[-1])

Rajasthan_data_df=Rajasthan_data
Rajasthan_data_df['route'] = Rajasthan_data['links'].apply(lambda x: x.split('/')[-1])

Telangana_data_df=Telangana_data
Telangana_data_df['route'] = Telangana_data['links'].apply(lambda x: x.split('/')[-1])

West_bengal_data_df=West_bengal_data
West_bengal_data_df['route'] = West_bengal_data['links'].apply(lambda x: x.split('/')[-1])

karbi_anglong_data_df=karbi_anglong_data
karbi_anglong_data_df['route'] = karbi_anglong_data['links'].apply(lambda x: x.split('/')[-1])

Kadamba_data_df=Kadamba_data
Kadamba_data_df['route'] = Kadamba_data['links'].apply(lambda x: x.split('/')[-1])

Bihar_data_df=Bihar_data
Bihar_data_df['route'] = Bihar_data['links'].apply(lambda x: x.split('/')[-1])


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# MySQL credentials
mysql_user = 'root'
mysql_password = '1234'
mysql_host = 'localhost'
mysql_port = '3306'
mysql_database = 'Redbus'

# Create a connection to MySQL database
engine = create_engine(f'mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{mysql_database}', echo=True)

# List of CSV file paths
csv_files = [
    'C:/Users/DhinU/Documents/Redbus/ap_data.csv', 
    'C:/Users/DhinU/Documents/Redbus/Bihar_data.csv', 
    'C:/Users/DhinU/Documents/Redbus/jammukashmir_data.csv', 
    'C:/Users/DhinU/Documents/Redbus/Kadamba_data.csv', 
    'C:/Users/DhinU/Documents/Redbus/Karbi_Anglong_data.csv',
    'C:/Users/DhinU/Documents/Redbus/Kerala_data.csv', 
    'C:/Users/DhinU/Documents/Redbus/Punjab_data.csv',
    'C:/Users/DhinU/Documents/Redbus/Rajasthan_data.csv',
    'C:/Users/DhinU/Documents/Redbus/Telangana_data.csv',
    'C:/Users/DhinU/Documents/Redbus/westbengal_data.csv'
]

# List to hold DataFrames
dataframes = []

# Loop through the list of CSV files
for file in csv_files:
    try:
        # Attempt to read the CSV file
        df = pd.read_csv(file)
        # Append the DataFrame to the list if successful
        dataframes.append(df)
    except pd.errors.EmptyDataError:
        print(f"Warning: {file} is empty and has been skipped.")
    except Exception as e:
        print(f"Error: Could not read {file} due to {e}")

# Concatenate all valid DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('combined_file.csv', index=False)

# Create a connection to MySQL database
engine = create_engine(f'mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{mysql_database}', echo=True)

#load the datasets int sql
combined_df.to_sql('combined', con=engine, index=False, if_exists='replace')